Code from OpenAI Cookbook
https://cookbook.openai.com/examples/chat_finetuning_data_prep

In [1]:
import json
import os
import tiktoken # for token counting
import numpy as np
from collections import defaultdict
import openai

Validate Data Format

In [2]:
# load jsonl file
data_path = "training_data.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 976
First example:
{'role': 'system', 'content': 'Imagine you are a member of the European\n Parliament and based on your years of experience, you are an expert in \n predicting how the different party groups will vote on a given law. Given\n the above legislative proposal, predict the percentage of votes in favour \n from each party group in the European Parliament. Assess the political \n direction, wording, framing, and topic relevance of the law to inform your \n predictions. Determine the type of majority (General, Left, Right, Consensus) \n likely to support the legislation based on party alignments.'}
{'role': 'user', 'content': '""Amendment of the MFF 2014-2020"": PURPOSE: to amend Regulation (EU, Euratom) No 1311/2013 laying down the multiannual financial framework for the years 2014-2020. PROPOSED ACT: Council Regulation. ROLE OF THE EUROPEAN PARLIAMENT: Council may adopt the act only if Parliament has given its consent to the act. CONTENT: Article 19 of Council

In [3]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [4]:
# HELPER FUNCTIONS

# cl100k_base is the tokenizer used by ChatGPT3.5 and ChatGPT4
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            # .encode returns the list of tokens (so that you can count them with len())
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [5]:
# check for missing data, distribution of messages in each conversation,
# distribution of tokens per conversation, print token limit warnings

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
#print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 85, 85
mean / median: 85.0, 85.0
p5 / p95: 85.0, 85.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


Estimate costs

In [6]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~969292 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~2907876 tokens


Upload validated files to OpenAI API

In [7]:
# upload validated data file to OpenAI API
# set OPENAI_API_KEY in environment variables
os.environ['TestKey3'] = 'sk-proj-GL73kbRwhRpgN3EmXz1YT3BlbkFJEMJhTsinxQDel42BZdNz' 

print(os.environ.get('TestKey3'))  

client = openai.OpenAI(api_key=os.environ['TestKey3'])

train_upload = client.files.create(
  file=open("small_training_data.jsonl", "rb"),
  purpose="fine-tune"
)
print("Uploaded training file id", train_upload.id)

val_upload = client.files.create(
  file=open("small_validation_data.jsonl", "rb"),
  purpose="fine-tune"
)
print("Uploaded validation file id", val_upload.id)

sk-proj-GL73kbRwhRpgN3EmXz1YT3BlbkFJEMJhTsinxQDel42BZdNz
Uploaded training file id file-vMVUI34iptTgrnBbM2arCQyZ
Uploaded validation file id file-QlbbKdDA7RNicmPPZfzXZKTZ


Create Fine-Tuning Job via OpenAI Software Development Kit

- use suffix parameter to name resulting model
- set additional parameters auch as batch size, learning rate, epochs, etc. to be found here: https://platform.openai.com/docs/api-reference/fine-tuning/create

In [8]:
learning_rate_multiplier = [0.1, 0.01, 0.001, 0.0001, 0.00001]

response = client.fine_tuning.jobs.create(
  training_file=train_upload.id, # file id returned after upload to API 
  validation_file=val_upload.id, # file id returned after upload to API
  model="gpt-3.5-turbo",
  suffix="small",
  seed=124,
  hyperparameters={
  "n_epochs": 3,
	"batch_size": 1,
	"learning_rate_multiplier": 0.01
  }
)
job_id = response.id

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {response.status}")

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(job_id)

# monitor training progress, get ft events
client.fine_tuning.jobs.list_events(
  fine_tuning_job_id=job_id,
  limit=10
)


Fine-tunning model with jobID: ftjob-ta18V6ve5x2fTmgpfFqhZxMp.
Training Response: FineTuningJob(id='ftjob-ta18V6ve5x2fTmgpfFqhZxMp', created_at=1721212187, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=0.01), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-URxBkHYInUDxHdJfjeVT2W58', result_files=[], seed=124, status='validating_files', trained_tokens=None, training_file='file-vMVUI34iptTgrnBbM2arCQyZ', validation_file='file-QlbbKdDA7RNicmPPZfzXZKTZ', estimated_finish=None, integrations=[], user_provided_suffix='small')
Training Status: validating_files


SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-fvlwF26iX0HxCoZY90fJARXC', created_at=1721212187, level='info', message='Validating training file: file-vMVUI34iptTgrnBbM2arCQyZ and validation file: file-QlbbKdDA7RNicmPPZfzXZKTZ', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-XCQPiLUg8doOc1XaT5nZqKzX', created_at=1721212187, level='info', message='Created fine-tuning job: ftjob-ta18V6ve5x2fTmgpfFqhZxMp', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

Cancel or track a fine-tune job or delete a model

In [ ]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-abc123")

# Cancel a job
client.fine_tuning.jobs.cancel("ftjob-abc123")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-abc123", limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

Using fine-tuned model via Playground or via code

In [ ]:
# make a request to the new model
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo:my-org:custom_suffix:id",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
print(completion.choices[0].message)

Metrics of the model

- while an active fine-tuning job is running via an event object
- or when job is finished via querying the job and retrieving result files as csv
- or by defining own eval method, see resources from OpenAI here: https://github.com/openai/evals
- OR use own classification model

In [ ]:
# Retrieving event object (ongoing job)
{
    "object": "fine_tuning.job.event",
    "id": "ftevent-abc-123",
    "created_at": 1693582679,
    "level": "info",
    "message": "Step 300/300: training loss=0.15, validation loss=0.27, full validation loss=0.40",
    "data": {
        "step": 300,
        "train_loss": 0.14991648495197296,
        "valid_loss": 0.26569826706596045,
        "total_steps": 300,
        "full_valid_loss": 0.4032616495084362,
        "train_mean_token_accuracy": 0.9444444179534912,
        "valid_mean_token_accuracy": 0.9565217391304348,
        "full_valid_mean_token_accuracy": 0.9089635854341737
    },
    "type": "metrics"
}